<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

### 1.1 - Common Config

In [1]:
from my_config import MY_CONFIG

### 1.2 - Inspect Input Data

We have a bnunch of datasets in [data](../data) folder.  Examine them

We will use one of them or feel free to bring your own data.

### 1.3 - Set input/output path variables for the pipeline

In [2]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### 1.4 -  Import Common python modules

In [3]:
import os
import sys


from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 2.1 -  Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='../data/papers' --> output='output-papers/01_parquet_out'


### 2.2 - Execute 

In [5]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")


14:33:21 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
14:33:21 INFO - pipeline id pipeline_id
14:33:21 INFO - code location None
14:33:21 INFO - data factory data_ is using local data access: input_folder - ../data/papers output_folder - output-papers/01_parquet_out
14:33:21 INFO - data factory data_ max_files -1, n_sample -1
14:33:21 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
14:33:21 INFO - orchestrator pdf2parquet started at 2024-09-20 14:33:21
14:33:21 INFO - Number of files is 2, source profile {'max_file_size': 5.398899078369141, 'min_file_size': 2.112621307373047, 'total_file_size': 7.5115203857421875}
14:33:21 INFO - Initializing models


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

14:33:55 INFO - Completed 1 files (50.0%) in 0.499 min
14:34:12 INFO - Completed 2 files (100.0%) in 0.79 min
14:34:12 INFO - Done processing 2 files, waiting for flush() completion.
14:34:12 INFO - done flushing in 0.0 sec
14:34:12 INFO - Completed execution in 0.854 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 1min 51s, sys: 1.79 s, total: 1min 52s
Wall time: 54.6 s


### 2.3 -  Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention-is-all-you-need.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",15,4,193,802bded3-330b-48d7-bb52-1098500decec,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:34:12.950537,17.431519,attention-is-all-you-need.pdf
1,Granite_Foundation_Models.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 3.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output-papers/01_parquet_out' --> output='output-papers/02_chunk_out'


### 3.2 - Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_python import DocChunkPythonTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(DocChunkPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

14:34:14 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
14:34:14 INFO - pipeline id pipeline_id
14:34:14 INFO - code location None
14:34:14 INFO - data factory data_ is using local data access: input_folder - output-papers/01_parquet_out output_folder - output-papers/02_chunk_out
14:34:14 INFO - data factory data_ max_files -1, n_sample -1
14:34:14 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:34:14 INFO - orchestrator doc_chunk started at 2024-09-20 14:34:14
14:34:14 INFO - Number of files is 2, source profile {'max_file_size': 0.0879373550415039, 'min_file_size': 0.03559398651123047, 'total_file_size': 0.12353134155273438}
14:34:

✅ Stage:2 completed successfully
CPU times: user 1.04 s, sys: 169 ms, total: 1.21 s
Wall time: 1.45 s


### 3.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 266
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (266, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
18,attention-is-all-you-need.pdf,15,4,193,802bded3-330b-48d7-bb52-1098500decec,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:34:12.950537,17.431519,attention-is-all-you-need.pdf,3 Model Architecture\nFigure 1: The Transforme...,$.main-text[30],3,"[209.53666687, 377.54708862, 401.99035645, 387..."
201,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,VI. SOCIO-TECHNICAL HARMS AND RISKS\nNumerous ...,$.main-text[219],11,"[47.95080566, 281.84732056, 300.52746582, 363...."
93,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,A. Overview of Capabilities\nThe 13b in the na...,$.main-text[9],1,"[48.07516098, 43.18538284, 300.03152466, 65.59..."


## Step-4:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 4.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output-papers/02_chunk_out' --> output='output-papers/03_docid_out'


### 4.2 - Execute 

In [11]:
%%time 

from doc_id_transform_python import DocIDPythonTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = PythonTransformLauncher(DocIDPythonTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:34:14 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
14:34:14 INFO - pipeline id pipeline_id
14:34:14 INFO - code location None
14:34:14 INFO - data factory data_ is using local data access: input_folder - output-papers/02_chunk_out output_folder - output-papers/03_docid_out
14:34:14 INFO - data factory data_ max_files -1, n_sample -1
14:34:14 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:34:14 INFO - orchestrator doc_id started at 2024-09-20 14:34:14
14:34:14 INFO - Number of files is 2, source profile {'max_file_size': 0.04591846466064453, 'min_file_size': 0.024618148803710938, 'total_file_size': 0.07053661346435547}
14:34:14 INFO - Completed 1 files (50.0%) in 0.0 min
14:34:14 INFO - Completed 2 files (100.0%) in 0.0 min
14:34:14 INFO - Done processing 2 files, waiting for flush() com

✅ Stage:3 completed successfully
CPU times: user 18.9 ms, sys: 4.39 ms, total: 23.3 ms
Wall time: 19.6 ms


### 4.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 15)
Output data dimensions (rows x columns)=  (266, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id
28,attention-is-all-you-need.pdf,15,4,193,802bded3-330b-48d7-bb52-1098500decec,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:34:12.950537,17.431519,attention-is-all-you-need.pdf,3.2.1 Scaled Dot-Product Attention\nWhile for ...,$.main-text[49],4,"[107.04788208, 174.61242676, 504.51269531, 219...",a1172fbaaef42850d7c9340740acd47badd2ce11b6f20a...,206
238,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,C. Evaluation\n1) Academic Benchmarks: We eval...,$.main-text[396],16,"[311.16088867, 43.3947525, 563.29870605, 89.57...",c92d1e14b79373b58950b72ad254fb01104403c6abb214...,150
217,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,"REFERENCES\n[1] J. Wei, M. Bosma, V. Y. Zhao, ...",$.main-text[250],13,"[145.87217712, 195.048172, 202.44746399, 204.7...",31b2b97050b77f7dfa0c1b0050fdb5f53c41e7ff45b53b...,129


## Step-5: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 5.1 - Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output-papers/03_docid_out' --> output='output-papers/04_exact_dedupe_out'


### 5.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_python import EdedupPythonTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # ededup parameters
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(EdedupPythonTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:34:14 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None}
14:34:14 INFO - pipeline id pipeline_id
14:34:14 INFO - code location None
14:34:14 INFO - data factory data_ is using local data access: input_folder - output-papers/03_docid_out output_folder - output-papers/04_exact_dedupe_out
14:34:14 INFO - data factory data_ max_files -1, n_sample -1
14:34:14 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:34:14 INFO - orchestrator ededup started at 2024-09-20 14:34:14
14:34:14 INFO - Number of files is 2, source profile {'max_file_size': 0.053391456604003906, 'min_file_size': 0.02887439727783203, 'total_file_size': 0.08226585388183594}
14:34:14 INFO - Starting from the beginning
14:34:14 INFO - Completed 1 files (50.0%) in 0.0 min
14:34:14 INFO - Completed 2 files (100.0%) in 0.0 min
14

✅ Stage:4 completed successfully
CPU times: user 19.2 ms, sys: 1.88 ms, total: 21.1 ms
Wall time: 17.8 ms


### 5.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 17)
Output data dimensions (rows x columns)=  (266, 18)
Input chunks before exact dedupe : 266
Output chunks after exact dedupe : 266
Duplicate chunks removed :   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id,removed
225,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,A. Model Outputs from Safety Tasks\nDisclaimer...,$.main-text[376],16,"[48.15233612, 488.89440918, 300.33282471, 547....",dc8577aca3ceb1e285464d661766a434f6fe8d1872d4c0...,137,[]
144,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,C. Energy Consumption and Carbon Emissions\nTh...,$.main-text[132],6,"[48.11833191, 383.87985229, 300.37054443, 478....",0a534e6e106839971a4b33caf6cfad192a3636cc246df9...,56,[]
131,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,A. Algorithmic Details\nThe granite.13b.v1 bas...,$.main-text[107],4,"[311.14950562, 43.96509933, 563.44586182, 65.6...",9fe2656e1b1b1057b9579fa94536926c8f07752ef37fc5...,43,[]


## Step-6: Fuzzy Dedup

**Fuzzy dedupe is currently available in RAY version**

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 6.1 - Set Input/output Folder

In [16]:
STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output-papers/04_exact_dedupe_out' --> output='output-papers/05_fuzzy_dedupe_out'


### 6.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher


# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

14:34:14 INFO - Running locally
14:34:14 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
14:34:14 INFO - data factory data_ is using local data access: input_folder - output-papers/04_exact_dedupe_out output_folder - output-papers/05_fuzzy_dedupe_out
14:34:14 INFO - data factory data_ max_files -1, n_sample -1
14:34:14 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:34:14 INFO - pipeline id pipeline_id
14:34:14 INFO - code location None
14:34:14 I

✅ Stage:5 completed successfully
CPU times: user 465 ms, sys: 348 ms, total: 813 ms
Wall time: 23.3 s


### 6.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 18)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash
258,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,JAPANESE FINANCE BENCHMARK EVALUATION RESULTS\...,$.main-text[432],18,"[213.05499268, 244.48545837, 398.37347412, 261...",170,[],-1
172,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,B. Granite Model Evaluation and Comparison\nEv...,$.main-text[180],8,"[48.16522598, 108.47112274, 300.1288147, 130.1...",84,[],-1
113,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,A. Data Clearance and Acquisition\nFig. 3. IBM...,$.main-text[60],3,"[311.41522217, 586.98126221, 457.3286438, 595....",25,[],-1


## Step-7:   Text encoding

Encode text for the vector storage.

### 7.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output-papers/05_fuzzy_dedupe_out' --> output='output-papers/06_embeddings_out'


### 7.2 - Execute

In [20]:
%%time 

from text_encoder_transform_python import TextEncoderPythonTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = PythonTransformLauncher(TextEncoderPythonTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

14:34:38 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
14:34:38 INFO - pipeline id pipeline_id
14:34:38 INFO - code location None
14:34:38 INFO - data factory data_ is using local data access: input_folder - output-papers/05_fuzzy_dedupe_out output_folder - output-papers/06_embeddings_out
14:34:38 INFO - data factory data_ max_files -1, n_sample -1
14:34:38 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
14:34:38 INFO - orchestrator text_encoder started at 2024-09-20 14:34:38
14:34:38 INFO - Number of files is 2, source profile {'max_file_size': 0.04749584197998047, 'min_file_size': 0.02600860595703125, 'total_file_size': 0.07350444793701172}
14:34:41 INFO - Completed 1 files (50.0%) in 0.016 min
14:34:42 INFO - Completed 2 files (100.0%) in 0.023

✅ Stage:6 completed successfully
CPU times: user 1.94 s, sys: 220 ms, total: 2.16 s
Wall time: 4.01 s


### 7.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 19)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,embeddings
210,Granite_Foundation_Models.pdf,20,13,444,7a91936e-f7b4-4d90-9278-9ec1d2a19475,pdf,3fdea13fde26d449edec114b8a7b70c4565673aa90c86b...,394434,2024-09-20T14:33:55.489893,29.919255,Granite_Foundation_Models.pdf,B. Downstream Documentation\nFor downstream us...,$.main-text[236],11,"[311.23956299, 62.55028915, 563.27984619, 84.1...",122,[],-1,"[-0.06926964, -0.09363627, 0.01313181, 0.01689..."
62,attention-is-all-you-need.pdf,15,4,193,802bded3-330b-48d7-bb52-1098500decec,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:34:12.950537,17.431519,attention-is-all-you-need.pdf,6.1 Machine Translation\nOn the WMT 2014 Engli...,$.main-text[108],8,"[107.27262115, 260.13467407, 505.24533081, 302...",240,[],-1,"[-0.03620785, -0.04418195, 0.02379151, -0.0163..."
54,attention-is-all-you-need.pdf,15,4,193,802bded3-330b-48d7-bb52-1098500decec,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-20T14:34:12.950537,17.431519,attention-is-all-you-need.pdf,5.3 Optimizer\nWe used the Adam optimizer [20]...,$.main-text[95],7,"[107.17208862, 193.10830688, 504.16403198, 215...",232,[],-1,"[0.037252318, -0.0319632, 0.0018505336, -0.037..."


## Step-8: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output-papers/06_embeddings_out' --> 'output-papers/output_final'
